In [2]:
# Setup from initial notebook setup. Run once!

# Duckdb is a SQL engine that allows us to execute powerful, analytics-friendly
# queries against local or remote databases and flat files.
import duckdb
import pandas as pd
from IPython.display import display # to adjust Pandas dataframe layouts
import matplotlib.pyplot as plt # for plotting

# Create a database file on disk
conn = duckdb.connect('example.db')
# Enable remote access
conn.sql("INSTALL httpfs")
conn.sql("LOAD httpfs")
# This database file points to files totaling multiple GBs,
# but it's only about 300KB itself. The `ATTACH` command
# gives us access to views that sit on top of remote Parquet files.
try:
  conn.sql("ATTACH 'https://data.baseball.computer/dbt/bc_remote.db' (READ_ONLY)")
except duckdb.BinderException:
  # This command will fail if you run it more than once because it already exists,
  # in which case we don't need to do anything
  pass

conn.sql("USE bc_remote")
conn.sql("USE main_models")

# Connect to a local DB to store remote results
local = duckdb.connect(database = "weird-stats.db", read_only = False)

In [3]:
# First, we need to find innings where the first play was a home run or a double

inningsSince2000 = conn.sql("""
SELECT
    game_id,
    inning,
    frame,
    row_number() over (
        partition by game_id, inning, frame
        order by event_key asc) playSeq,
    plate_appearance_result
FROM
    event.events plays
WHERE
    CAST(substr(game_id,4,4) AS DOUBLE) > 2000

""").df()

local.sql("""CREATE OR REPLACE TABLE inningsSince2000 AS (
SELECT
    *
FROM
    inningsSince2000)
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
local.sql("""CREATE OR REPLACE VIEW firstPlaysSince2000 AS (
SELECT
    *
FROM
    inningsSince2000
WHERE
    playSeq=1
)
""")

In [5]:
local.sql("""SELECT * FROM firstPlaysSince2000""").df()

,game_id,inning,frame,playSeq,plate_appearance_result
0,DET200607150,2,Top,1,StrikeOut
1,DET200608080,8,Bottom,1,None
2,DET200608240,4,Top,1,Double
3,DET200609290,3,Bottom,1,HomeRun
4,DET200610130,7,Bottom,1,Walk
...,...,...,...,...,...
986509,WAS202309060,7,Bottom,1,Single
986510,WAS202309080,2,Bottom,1,InPlayOut
986511,WAS202309100,9,Bottom,1,None
986512,WAS202309180,8,Top,1,InPlayOut


In [6]:
local.sql("""CREATE OR REPLACE VIEW leadoffAnalysis AS (
SELECT
    *
FROM
    firstPlaysSince2000
-- WHERE
--     plate_appearance_result like '%Double'
--    OR plate_appearance_result like '%HomeRun'
ORDER BY
    substr(game_id,4,4) asc,
    substr(game_id,8,2) asc,
    substr(game_id,10,2) asc
)
""")

In [8]:
innings_analysis = local.sql("""SELECT * FROM leadoffAnalysis WHERE plate_appearance_result NOT LIKE '%Out'""").df()
innings_analysis

,game_id,inning,frame,playSeq,plate_appearance_result
0,TOR200104010,1,Top,1,HitByPitch
1,TOR200104010,5,Bottom,1,Single
2,TOR200104010,2,Bottom,1,ReachedOnError
3,TOR200104010,4,Top,1,Walk
4,FLO200104020,3,Bottom,1,Walk
...,...,...,...,...,...
239981,ARI202310310,1,Bottom,1,Single
239982,ARI202311010,7,Top,1,Single
239983,ARI202311010,3,Bottom,1,Single
239984,ARI202311010,2,Bottom,1,Single


In [21]:
inningsequence = conn.sql("""
SELECT
    plays.*,
    CASE 
        WHEN plays.plate_appearance_result LIKE '%Double'
            THEN 2
        WHEN plays.plate_appearance_result LIKE '%Triple'
            THEN 3
        WHEN plays.plate_appearance_result LIKE '%Single'
            THEN 1
        WHEN plays.plate_appearance_result LIKE '%HomeRun'
            THEN 4
        WHEN plays.plate_appearance_result LIKE '%Out'
            THEN -1
        ELSE 0
    END as bases,
    row_number() OVER (
        partition by game_id, inning, frame
        order by event_id asc) inningbatter
FROM
    event.events plays
    JOIN innings_analysis
        USING (game_id, inning, frame)

""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
local.sql("""CREATE OR REPLACE TABLE innings as (SELECT * FROM inningsequence)""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
local.sql("""CREATE OR REPLACE VIEW inning_grouped AS (
SELECT 
    game_id,
    inning,
    frame,
    array_agg(plate_appearance_result order by inningbatter asc)[1] firstresult,
    COUNT(distinct event_id) batters,
    SUM(bases^2) basesScore,
    SUM(outs_on_play^2) outsScore,
    SUM(runs_on_play) runsScore
FROM 
    INNINGS
GROUP BY
1,2,3
ORDER BY
    substr(game_id,4,4) asc,
    substr(game_id,8,2) asc,
    substr(game_id,10,2) asc
)
""")
local.sql("""SELECT * FROM inning_grouped""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,game_id,inning,frame,firstresult,batters,basesScore,outsScore,runsScore
0,TOR200104010,2,Bottom,ReachedOnError,4,3.0,3.0,0.0
1,TOR200104010,1,Top,HitByPitch,5,7.0,5.0,1.0
2,TOR200104010,4,Top,Walk,5,4.0,3.0,0.0
3,TOR200104010,5,Bottom,Single,6,8.0,3.0,1.0
4,COL200104020,1,Bottom,Single,7,22.0,3.0,2.0
...,...,...,...,...,...,...,...,...
239981,ARI202310310,1,Bottom,Single,4,3.0,3.0,0.0
239982,ARI202311010,1,Bottom,Walk,6,3.0,3.0,0.0
239983,ARI202311010,3,Bottom,Single,5,3.0,3.0,0.0
239984,ARI202311010,2,Bottom,Single,4,4.0,3.0,0.0


In [24]:
local.sql("""  CREATE VIEW IF NOT EXISTS netscores as (
with adjustedScores as (SELECT
    game_id,
    inning,
    frame,
    firstresult,
    batters-1 batters,
    basesScore - CASE
        WHEN firstresult like '%Double' THEN 2^2
        WHEN firstresult like '%Triple' THEN 3^2
        WHEN firstresult like '%HomeRun' THEN 4^2
        ELSE 1
    END BasesScore,
    outsScore,
    GREATEST(runsScore - 1,0) runsScore
FROM
    inning_grouped
)
SELECT
    *,
    batters+basesScore+runsScore-outsScore netScore
FROM
    adjustedScores
ORDER BY
    netScore desc)
""")

In [26]:
local.sql("""
SELECT 
    firstresult, 
    COUNT(*) records, 
    SUM(netScore) totalScore, 
    AVG(netScore) avgRallyScore
FROM
    netScores
WHERE (firstresult LIKE '%Double'
        OR firstresult LIKE '%Single'
        OR firstresult LIKE '%Triple'
        OR firstresult LIKE '%HomeRun'
        )
GROUP BY 1
ORDER BY 4 desc
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,firstresult,records,totalScore,avgRallyScore
0,InsideTheParkHomeRun,63,584.0,9.269841
1,HomeRun,23270,200213.0,8.603911
2,Triple,3763,31276.0,8.311454
3,Double,34418,278763.0,8.099338
4,GroundRuleDouble,2025,16209.0,8.004444
5,Single,113582,829207.0,7.300514
